# L331 ~ L340

> 所有题目著作权归领扣网络所有

## 331. 验证二叉树的前序序列化

序列化二叉树的一种方法是使用前序遍历。当我们遇到一个非空节点时，我们可以记录下这个节点的值。如果它是一个空节点，我们可以使用一个标记值记录，例如`#`

```
     _9_
    /   \
   3     2
  / \   / \
 4   1  #  6
/ \ / \   / \
# # # #   # #
```

例如，上面的二叉树可以被序列化为字符串`9,3,4,#,#,1,#,#,2,#,6,#,#`，其中`#`代表一个空节点  
给定一串以逗号分隔的序列，验证它是否是正确的二叉树的前序序列化。编写一个在不重构树的条件下的可行算法  
每个以逗号分隔的字符或为一个整数或为一个表示`null`指针的`#`  
你可以认为输入格式总是有效的，例如它永远不会包含两个连续的逗号，比如`"1,,3"`

解题思路：

我们可以定义一个概念，叫做槽位。一个槽位可以被看作「当前二叉树中正在等待被节点填充」的那些位置  
二叉树的建立也伴随着槽位数量的变化。每当遇到一个节点时：

- 如果遇到了空节点，则要消耗一个槽位；
- 如果遇到了非空节点，则除了消耗一个槽位外，还要再补充两个槽位

此外，还需要将根节点作为特殊情况处理

![](./asset/L331_t.png)

我们使用栈来维护槽位的变化。栈中的每个元素，代表了对应节点处**剩余槽位的数量**，而栈顶元素就对应着下一步可用的槽位数量。当遇到空节点时，仅将栈顶元素减 `1`；当遇到非空节点时，将栈顶元素减`1`后，再向栈中压入一个`2`。无论何时，如果栈顶元素变为`0`，就立刻将栈顶弹出  
遍历结束后，若栈为空，说明没有待填充的槽位，因此是一个合法序列；否则若栈不为空，则序列不合法。此外，在遍历的过程中，若槽位数量不足，则序列不合法


**示例1:**

```
输入: "9,3,4,#,#,1,#,#,2,#,6,#,#"
输出: true
```

**示例2:**

```
输入: "1,#"
输出: false
```

**示例3:**

```
输入: "9,#,#,1"
输出: false
```

### 331.1. Solution

- Sulution 1

In [ ]:
def is_valid_serialization1(preorder: str) -> bool:
    """
    算法思路:


    Args:
        preorder(str): 输入的二叉树前序序列

    Returns:
        bool: 输入的前序序列是否正确
    """
    i = 0
    stack = [1]
    while i < len(preorder):
        if len(stack) == 0:
            return False

        if preorder[i] == ',':
            i += 1
        elif preorder[i] == '#':  # 空节点
            stack[-1] -= 1
            if stack[-1] == 0:
                stack.pop()
            i += 1
        else:
            while i < len(preorder) and preorder[i] != ',':
                i += 1

            stack[-1] -= 1
            if stack[-1] == 0:
                stack.pop()

            stack.append(2)

    return len(stack) == 0

- Sulution 2

In [ ]:
def is_valid_serialization2(preorder: str) -> bool:
    """
    算法思路:
        和 Sulution 1 思路一致，将栈的使用改为计数器，代表栈中所有元素之和

    Args:
        preorder(str): 输入的二叉树前序序列

    Returns:
        bool: 输入的前序序列是否正确
    """
    i = 0
    slots = 1
    while i < len(preorder):
        if slots == 0:
            return False

        if preorder[i] == ',':
            i += 1
        elif preorder[i] == '#':  # 空节点
            slots -= 1
            i += 1
        else:
            while i < len(preorder) and preorder[i] != ',':
                i += 1
            slots += 1  # slots = slots - 1 + 2

    return slots == 0

- Sulution 3

In [ ]:
def is_valid_serialization3(preorder: str) -> bool:
    """
    算法思路:
        参考 fuxuemingzhu 的答案（https://leetcode-cn.com/problems/verify-preorder-serialization-of-a-binary-tree/solution/pai-an-jiao-jue-de-liang-chong-jie-fa-zh-66nt/）
        在树（甚至图）中，所有节点的入度之和等于出度之和。可以根据这个特点判断输入序列是否为有效的
        在一棵二叉树中：
            - 每个空节点（"#"）会提供 0 个出度和 1 个入度
            - 每个非空节点会提供 2 个出度和 1 个入度
            - 我们只要把字符串遍历一次，每个节点都累加 diff = 出度 - 入度
              在遍历到任何一个节点的时候，要求 diff >= 0，原因是还没遍历到该节点的子节点，所以此时的出度应该大于等于入度
              当所有节点遍历完成之后，整棵树的 diff == 0

        为什么下面的代码中 diff 的初始化为 1？
            因为，加入一个非空节点时，都会先减去一个入度，再加上两个出度。但是由于根节点没有父节点，所以其入度为 0，出度为 2
            因此 diff 初始化为 1，是为了在加入根节点的时候，先减去一个入度，再加上两个出度，此时 diff 正好应该是2

    Args:
        preorder(str): 输入的二叉树前序序列

    Returns:
        bool: 输入的前序序列是否正确
    """
    nodes = preorder.split(',')
    diff = 1
    for node in nodes:
        diff -= 1

        if diff < 0:
            return False
        if node != '#':
            diff += 2

    return diff == 0

### 331.2. Test

- Test 1

In [ ]:
preorder = '9,3,4,#,#,1,#,#,2,#,6,#,#'
res = is_valid_serialization1(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '1,#'
res = is_valid_serialization1(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '9,#,#,1'
res = is_valid_serialization1(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

- Test 2

In [ ]:
preorder = '9,3,4,#,#,1,#,#,2,#,6,#,#'
res = is_valid_serialization2(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '1,#'
res = is_valid_serialization2(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '9,#,#,1'
res = is_valid_serialization2(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

- Test 3

In [ ]:
preorder = '9,3,4,#,#,1,#,#,2,#,6,#,#'
res = is_valid_serialization3(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '1,#'
res = is_valid_serialization3(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))

preorder = '9,#,#,1'
res = is_valid_serialization3(preorder=preorder)
print('The serialization "{}" is valid? {}'.format(preorder, res))